<a href="https://colab.research.google.com/github/NatDar/-Python/blob/main/DNA_NLP_HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download ru_core_news_lg

2023-11-02 16:45:21.684788: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 16:45:21.684852: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 16:45:21.684886: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 16:45:21.692537: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 16:45:22.786225: W tensorflow/c

In [2]:
pip install corus razdel navec slovnet deeppavlov

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.2/489.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Succ

In [3]:
!mkdir datasets
!wget https://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip -d ./datasets

--2023-11-02 16:48:52--  https://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.75MB/s    in 1.0s    

2023-11-02 16:48:54 (1.75 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: ./datasets/Collection5/
  inflating: ./datasets/Collection5/001.ann  
  inflating: ./datasets/Collection5/001.txt  
  inflating: ./datasets/Collection5/002.ann  
  inflating: ./datasets/Collection5/002.txt  
  inflating: ./datasets/Collection5/003.ann  
  inflating: ./datasets/Collection5/003.txt  
  inflating: ./datasets/Collection5/004.ann  
  inflating: ./datasets/Collection5/004.txt  
  inflating: ./datasets/Collection5/005.ann  
  inflating: .

In [5]:
import nltk
import os
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import spacy
from spacy import displacy
import ru_core_news_lg

import corus
from corus import load_ne5
from razdel import tokenize

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [6]:
dataset_for_nltk = []

for filename in os.scandir("datasets/Collection5/"):
    if filename.is_file():
        name, ext = os.path.splitext(filename.path)
        if ext == ".txt":
          with(open(filename.path,"r",encoding='utf-8') as f):
            text = f.read()
            tokens = text.split()

          tagged_tokens = {}

          with(open(name + ".ann","r",encoding='utf-8') as f):
            for line in f:
              parts = line.split()
              tag = parts[1]
              name = ' '.join(parts[4:])
              tagged_tokens[name] = tag

          dataset_for_nltk.append((text, tagged_tokens))


In [7]:
for entry in dataset_for_nltk:
  pred = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(entry[0]))) if hasattr(chunk, 'label')]
  print(entry[1])
  print(pred)
  break

# Работает довольно плохо

{'Кремль': 'GEOPOLIT', 'Путина': 'PER', 'Сердюковым': 'PER', 'Петербурге': 'LOC', 'Дмитрий Песков': 'PER', 'Владимир Путин': 'PER', 'Минобороны': 'ORG', 'Анатолия Сердюкова': 'PER', 'МК': 'MEDIA', 'РФ': 'GEOPOLIT', 'Анатолием Сердюковым': 'PER', 'РИА Новости': 'MEDIA', 'Песков': 'PER', 'Московский комсомолец': 'MEDIA', 'Анатолий Сердюков': 'PER', 'Оборонсервиса': 'ORG', 'СМИ': 'MEDIA', 'Ростехнологий': 'ORG', 'Сергея Чемезова': 'PER', 'Сердюкова': 'PER', 'Сердюков': 'PER', 'Роснано': 'ORG', 'Анатолия Чубайса': 'PER', 'Константин Богданов': 'PER'}
[('Кремль', 'PERSON'), ('Путина', 'PERSON'), ('Дмитрий Песков', 'PERSON'), ('Дмитрий Песков', 'PERSON'), ('Минобороны Анатолием Сердюковым', 'ORGANIZATION'), ('Новости Песков', 'PERSON'), ('Путина', 'PERSON'), ('Анатолий Сердюков', 'PERSON'), ('Новости', 'PERSON'), ('Сергея Чемезова', 'PERSON'), ('Анатолия Чубайса', 'PERSON'), ('РИА Новости Константин Богданов', 'ORGANIZATION'), ('Сердюкова', 'PERSON'), ('Возможно', 'PERSON')]


In [9]:
nlp = ru_core_news_lg.load()
doc = nlp(dataset_for_nltk[0][0])
displacy.render(doc, jupyter=True, style='ent')

In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [11]:
!locale

LANG=en_US.UTF-8
LANGUAGE=en_US
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=en_US.UTF-8


In [12]:
def load_text_patched(path):
    # do not convert \r\n to \n
    with open(path, newline='', encoding='utf-8') as file:
        return file.read()

dir = 'datasets/Collection5/'
# load_ne5 do not accept encoding, but Colab sometimes fails to default to utf-8 for unknown reasons
corus.ne5.load_text = load_text_patched
records = load_ne5(dir)

In [13]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [15]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [16]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [17]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [18]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [19]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 128

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 5),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [20]:
embedding_dim = 128

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)

        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)

        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [21]:
model = modelNER()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['acc'])

In [22]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 93s 7ms/step - loss: 0.2838 - acc: 0.9174 - val_loss: 0.2108 - val_acc: 0.9375
Epoch 2/5
12444/12444 [==============================] - 74s 6ms/step - loss: 0.1214 - acc: 0.9636 - val_loss: 0.2161 - val_acc: 0.9387
Epoch 3/5
12444/12444 [==============================] - 76s 6ms/step - loss: 0.1070 - acc: 0.9664 - val_loss: 0.2238 - val_acc: 0.9398
Epoch 4/5
12444/12444 [==============================] - 74s 6ms/step - loss: 0.1021 - acc: 0.9671 - val_loss: 0.2652 - val_acc: 0.8823
Epoch 5/5
12444/12444 [==============================] - 75s 6ms/step - loss: 0.0993 - acc: 0.9679 - val_loss: 0.2338 - val_acc: 0.9373


In [23]:
y_pred1 = model.predict(valid_x)
y_pred = np.argmax(y_pred1, axis=1)

print(f'Precision score {precision_score(valid_y, y_pred , average="macro")}')
print(f'Recall score {recall_score(valid_y, y_pred , average="macro")}')
print(f'F1 score {f1_score(valid_y, y_pred , average="macro")}')

2074/2074 [==============================] - 7s 3ms/step
Precision score 0.9210208072262619
Recall score 0.7472537838745952
F1 score 0.8145342743020326


Одно слово (5 эпох):  
Train Accuracy: 0.9678  
Val Accuracy: 0.9387  
Precision: 0.9188477989540954  
Recall: 0.7682516646226318  
F1: 0.8281895170964603  
  
С (1,5)-грамами (3 эпохи):  
Train Accuracy: 0.9657  
Val Accuracy: 0.9412  
Precision: 0.9226781221438194  
Recall: 0.7843320758632909  
F1: 0.8371973641394906